In [57]:
!pip install "langchain==0.3.*" "langgraph>=0.2.0" google-generativeai langchain-google-genai altair

In [58]:
import os
import json
from typing import Dict, Union, List, Any

import altair as alt
import pandas as pd

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage


### Load dataset preprocessed in the data_processing ipynb file

In [ ]:
YU_paper = pd.read_csv('/scratch/Jli/Assignment/yeshiva_subset/papers_yeshiva.csv')
YU_fields = pd.read_csv('/scratch/Jli/Assignment/yeshiva_subset/fields_yeshiva.csv')
YU_refs = pd.read_csv('/scratch/Jli/Assignment/yeshiva_subset/refs_yeshiva.csv')
YU_affils = pd.read_csv('/scratch/Jli/Assignment/yeshiva_subset/affils_yeshiva.csv')


In [ ]:
DATA_DIR = "/scratch/Jli/Assignment/sciscinet-v2"

sciscinet_fields = pd.read_parquet(os.path.join(DATA_DIR, "sciscinet_fields.parquet"))
institutions = pd.read_parquet(os.path.join(DATA_DIR, "sciscinet_affiliations.parquet"))
sciscinet_fields.head()
institutions.head()

,institution_id,display_name,ror,homepage_url,country_code,type,h_index,productivity,avg_c10,avg_logc10
0,I4210136388,Carnot Institute Qualiment,032k75171,https://www.instituts-carnot.eu/en/carnot-inst...,FR,facility,NaN,NaN,NaN,NaN
1,I4210163148,Korean Institute of Southeast Asian Studies,05gkpvg94,http://www.kiseas.org/,KR,nonprofit,0.0,1.0,0.0,0.000000
2,I4210145380,Westlake Porter Public Library,049kpjn93,https://www.westlakelibrary.org/,US,archive,1.0,1.0,3.0,1.386294
3,I4210107623,Secretaria do Governo Municipal,01psywt28,http://www.prefeitura.sp.gov.br/cidade/secreta...,BR,government,NaN,NaN,NaN,NaN
4,I1293062124,Automotive Service Excellence,04qby6e62,https://www.ase.com/Home.aspx,US,nonprofit,NaN,NaN,NaN,NaN


In [ ]:
print(YU_paper.head())
print("--------------------------Field-------------------------------")
print(YU_fields.head())
print("--------------------------Refs--------------------------------")
print(YU_refs.head())
print("--------------------------Affilis-----------------------------")
print(YU_affils.head())

       paperid                                         doi  year        date  \
0  W4230191490   https://doi.org/10.1080/07908318909525055  1989  1989-01-01   
1  W4236999369    https://doi.org/10.1515/9783111684376.fm  1978  1978-12-31   
2  W4246924358  https://doi.org/10.1142/9789814295895_0001  2010  2010-08-01   
3   W765639894       https://doi.org/10.1300/j358v04n01_02  1988  1988-04-25   
4  W1992119014  https://doi.org/10.1093/jaarel/xxxix.2.254  1971  1971-01-01   

        doctype  cited_by_count  is_retracted  reference_count  \
0       article               0         False                6   
1  book-chapter               0         False                0   
2       article               0         False                0   
3       article               0         False                0   
4       article               0         False                0   

   citation_count  C3  ...  WSB_Cinf  SB_B  SB_T  team_size  \
0               0   0  ...       NaN   NaN   NaN          0

In [18]:
Paper_fields = YU_fields.merge(
    sciscinet_fields[["fieldid", "display_name"]],
    on="fieldid",
    how="left"
)

Paper_fields.head()

,paperid,fieldid,score_openalex,display_name
0,W4230191490,C144024400,0.456282,Sociology
1,W4230191490,C29595303,0.413342,Media studies
2,W4230191490,C17744445,0.412701,Political science
3,W4230191490,C142362112,0.265091,Art
4,W4230191490,C52119013,0.219945,Art history


In [ ]:
Paper_w_fields = YU_paper.merge(
    Paper_fields[["paperid", "fieldid", "display_name", "score_openalex"]],
    on="paperid",
    how="left"
)

columns = ["paperid", "doi", "year", "date", "doctype", "cited_by_count", "is_retracted", "reference_count", "citation_count", "fieldid", "display_name", "score_openalex"]
Paper_w_fields = Paper_w_fields[columns]

Paper_w_fields = Paper_w_fields.rename(
    columns={
        "display_name": "field",
        "institution": "institution",
    }
)
Paper_w_fields.head()



,paperid,doi,year,date,doctype,cited_by_count,is_retracted,reference_count,citation_count,fieldid,display_name,score_openalex
0,W4230191490,https://doi.org/10.1080/07908318909525055,1989,1989-01-01,article,0,False,6,0,C144024400,Sociology,0.456282
1,W4230191490,https://doi.org/10.1080/07908318909525055,1989,1989-01-01,article,0,False,6,0,C29595303,Media studies,0.413342
2,W4230191490,https://doi.org/10.1080/07908318909525055,1989,1989-01-01,article,0,False,6,0,C17744445,Political science,0.412701
3,W4230191490,https://doi.org/10.1080/07908318909525055,1989,1989-01-01,article,0,False,6,0,C142362112,Art,0.265091
4,W4230191490,https://doi.org/10.1080/07908318909525055,1989,1989-01-01,article,0,False,6,0,C52119013,Art history,0.219945


In [40]:
Paper_field_inst = Paper_w_fields.merge(
    YU_affils[["paperid", "author_position", "authorid", "institutionid"]],
    on="paperid",
    how="left"
)

Paper_field_inst = Paper_field_inst.rename(columns={
        "institutionid": "institution_id",
        "display_name": "field"
    })

Paper_field_inst = Paper_field_inst.merge(
    institutions[["institution_id", "display_name"]],
    on="institution_id",
    how="left"
)

Paper_field_inst = Paper_field_inst.rename(columns={
        "institutionid": "institution_id",
        "display_name": "institution"
    })

Paper_field_inst.head()

Paper_field_inst["year"] = Paper_field_inst["year"].astype(int)
Paper_field_inst.head()

,paperid,doi,year,date,doctype,cited_by_count,is_retracted,reference_count,citation_count,fieldid,field,score_openalex,author_position,authorid,institution_id,institution
0,W4230191490,https://doi.org/10.1080/07908318909525055,1989,1989-01-01,article,0,False,6,0,C144024400,Sociology,0.456282,last,A5108089204,I19772626,Yeshiva University
1,W4230191490,https://doi.org/10.1080/07908318909525055,1989,1989-01-01,article,0,False,6,0,C29595303,Media studies,0.413342,last,A5108089204,I19772626,Yeshiva University
2,W4230191490,https://doi.org/10.1080/07908318909525055,1989,1989-01-01,article,0,False,6,0,C17744445,Political science,0.412701,last,A5108089204,I19772626,Yeshiva University
3,W4230191490,https://doi.org/10.1080/07908318909525055,1989,1989-01-01,article,0,False,6,0,C142362112,Art,0.265091,last,A5108089204,I19772626,Yeshiva University
4,W4230191490,https://doi.org/10.1080/07908318909525055,1989,1989-01-01,article,0,False,6,0,C52119013,Art history,0.219945,last,A5108089204,I19772626,Yeshiva University


In [41]:
OUT_DIR = "/scratch/Jli/Assignment/yeshiva_subset/"
os.makedirs(OUT_DIR, exist_ok=True)

Paper_field_inst.to_csv(os.path.join(OUT_DIR, "Paper_field_inst_yu.csv"), index=False)